In [305]:
import pandas as pd
import numpy as np
import re
kilid = pd.read_csv("../Data_collecting/crawlers/kilid/kilid.csv")

In [306]:
combined_facilities = []
kilid.facilities.fillna("none", inplace=True)
kilid.facilities = kilid.facilities.str.split(",").values

convert_fac = {'آسانسور': "elevator", 
'آنتن مرکزی': "central antenna",
'استخر': "pool",
'انباری': "warehouse",
'بالکن': "balcony",
'تهویه مطبوع': "air condition",
'جکوزی': "jacuzzi",
'درب ریموت': "remote door",
'روف گاردن': "roof garden",
'سالن اجتماعات': "conference hall",
'سالن ورزش': "sports hall",
'سونا': "sauna",
'قدرالسهم': "qadrosahm",
'قیمت توافقی': "tavafoghi price",
'لابی': "lobby",
'مال': "mall",
'پاساژ': "mall",
'مشارکتی': "collaborative",
'معاوضه': "changable",
'موقعیت اداری': "administrative position",
'نگهبان': "guard",
'وام دار': "have loan",
'نوساز': "new built",
'پیش فروش': "preorder",
}
kilid['facilities'] = kilid['facilities'].apply(lambda x: [convert_fac.get(item, item) for item in x])




In [307]:
kilid = kilid.drop(columns= ["date", "floor", "func", "type", "title", "address", "desc", "properties"])
kilid

,age,bedroom,facilities,latitude,longitude,meterage,parking,price,price_per_meter
0,"1, 5",2,"[balcony, warehouse, remote door, elevator]",35.744625,51.294935,"7, 2",1,5.8 میلیارد,NaN
1,0,2,"[sauna, warehouse, roof garden, balcony, remot...",35.750249,51.288913,"8, 5",1,11.1 میلیارد,NaN
2,"1, 8",2,"[elevator, warehouse, balcony, guard]",35.721644,51.348070,"1, 0, 0",1,8.6 میلیارد,NaN
3,7,1,"[warehouse, balcony, elevator, guard]",35.720405,51.299933,"5, 5",1,5.2 میلیارد,NaN
4,"1, 8",1,"[remote door, warehouse, elevator]",35.724644,51.300270,"5, 3",1,5 میلیارد,95 میلیون
...,...,...,...,...,...,...,...,...,...
18961,"1, 8",2,"[balcony, remote door, air condition, elevator...",35.731491,51.287720,"7, 1",1,5.8 میلیارد,NaN
18962,8,2,"[warehouse, elevator, balcony, remote door, ai...",35.739790,51.289623,"7, 2",1,5.6 میلیارد,NaN
18963,"1, 8",1,"[balcony, warehouse, elevator, remote door, ha...",35.736118,51.297729,"6, 5",1,6 میلیارد,NaN
18964,"1, 8",1,"[warehouse, air condition, remote door, balcon...",35.731491,51.287720,"5, 5",1,4.1 میلیارد,NaN


In [308]:
def clean_and_convert(value):
    re.findall(r'[\d.]+', value)
    value = value.replace(', ', '')
    value = value.replace('ساله', '0')
    value = re.sub(r'[^0-9,]', '', value)

    return float(value) if value.isdigit() else 0

In [309]:
kilid.meterage = pd.to_numeric(kilid.meterage.str.replace(", ", ""))
kilid.age = kilid.age.apply(clean_and_convert)
kilid.price = kilid.price.apply(clean_and_convert)


In [312]:
kilid.price = (kilid.price*10).astype(int)


In [329]:
kilid[["price"]].sort_values(by="price")
kilid[kilid["price"] > 1000000]


,age,bedroom,facilities,latitude,longitude,meterage,parking,price,price_per_meter
5331,18.0,2,"[balcony, remote door, lobby, warehouse, eleva...",35.787802,51.460655,110,1,1530000,NaN
5768,0.0,4,"[lobby, jacuzzi, guard, warehouse, elevator, p...",35.790380,51.421093,4002,4,1560800,390 میلیون
12552,0.0,3,"[air condition, lobby, elevator, sauna, wareho...",35.818698,51.467745,245,3,6860000,280 میلیارد
13666,8.0,2,"[central antenna, elevator, balcony, warehouse...",35.717889,51.332495,98,1,1136800,11.6 میلیارد
16990,0.0,4,"[air condition, pool, sports hall, lobby, elev...",35.813351,51.405371,390,3,15210000,390 میلیارد


In [270]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

X = kilid[["latitude", "longitude"]]
kmeans = KMeans(n_clusters=15, random_state=42, n_init="auto")
kmeans.fit(X)
labels = kmeans.labels_
centroids = kmeans.cluster_centers_
kilid["cluster"] = labels

plt.plot(kind = "scatter", kilid['longitude'], kilid['latitude'], c=labels, s = 10, cmap='jet', marker='o', label='Houses', figsize = )
plt.plot(centroids[:, 1], centroids[:, 0], marker='X', s=100, c='black', label='Centroids')
plt.title('K-means Clustering of Houses with Centroids')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend()
plt.show()



SyntaxError: positional argument follows keyword argument (3097540149.py, line 12)

In [ ]:
kilid.sort_values(by="latitude")

In [213]:
# from sklearn.cluster import MiniBatchKMeans
# import matplotlib.pyplot as plt
# from sklearn.preprocessing import StandardScaler

# # Assuming 'kilid' is your DataFrame with columns 'latitude' and 'longitude'

# # Feature scaling if necessary
# scaler = StandardScaler()
# kilid[['latitude', 'longitude']] = scaler.fit_transform(kilid[['latitude', 'longitude']])

# # Determine optimal number of clusters (K)
# # Use the Elbow Method or Silhouette Score
# # For example, using the Elbow Method
# inertia = []
# for k in range(1, 20):
#     kmeans = KMeans(n_clusters=k, random_state=42)
#     kmeans.fit(kilid[['latitude', 'longitude']])
#     inertia.append(kmeans.inertia_)

# plt.plot(range(1, 20), inertia, marker='o')
# plt.xlabel('Number of Clusters (K)')
# plt.ylabel('Inertia')
# plt.title('Elbow Method')
# plt.show()

# # Based on the Elbow Method, choose an appropriate K value

# # Perform K-means clustering with the chosen K
# k = 20  # For example, after observing the elbow point
# kmeans = KMeans(n_clusters=k, random_state=42)
# kmeans.fit(kilid[['latitude', 'longitude']])
# labels = kmeans.labels_
# kilid['cluster'] = labels

# # Visualize clusters on a scatter plot
# plt.scatter(kilid['longitude'], kilid['latitude'], c=labels, cmap='viridis', marker='D')
# plt.title('K-means Clustering of Houses')
# plt.xlabel('Longitude')
# plt.ylabel('Latitude')
# plt.show()


0.49924343322075837